In [2]:
import pandas as pd
import numpy as np
import re 
import matplotlib.pyplot as plt
import seaborn 
import scipy
import sklearn
%matplotlib inline

In [3]:
import feather
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [4]:
import xgboost as xgb
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss

/Users/omardiaz/.pyenv/versions/3.5.2/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
walmart_df = feather.read_dataframe('walmart.feather')
nrow=walmart_df['TripType'].count()
#walmart_df_Y=walmart_df['TripType']
#walmart_df=walmart_df.drop(['VisitNumber', 'TripType'], axis=1)

In [6]:
walmart_df['Weekday'] = walmart_df['Weekday'].map({"Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4, "Friday": 5, 
                                           "Saturday": 6, "Sunday": 7})

In [7]:

walmart_df['TripType'] = walmart_df['TripType'].map({3:0, 4:1, 5:2, 6:3, 7:4, 8:5, 9:6, 12:7, 14:8, 15:9,  
                                         18:10, 19:11, 20:12, 21:13, 22:14, 23:15, 24:16, 25:17, 26:18,  
                                         27:19, 28:20, 29:21, 30:22, 31:23, 32:24, 33:25, 34:26, 35:27, 
                                         36:28, 37:29, 38:30, 39:31, 40:32, 41:33, 42:34, 43:35, 44:36, 999:37})
walmart_df = walmart_df.set_index('VisitNumber')

In [8]:
walmart_df.head()

,TripType,Weekday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,...,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS,total_prod,ind_dev,dia_mes,departamentos_visitados,prod_distintos_upc,prod_devueltos
VisitNumber,,,,,,,,,,,,,,,,,,,,,
5,37,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-1.0,1.0,1.0,1,0.0,1.0
7,22,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,1.0,2,2.0,0.0
8,18,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,28.0,1.0,1.0,7,21.0,2.0
9,5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,1.0,2,3.0,0.0
10,5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,1.0,2,3.0,0.0


In [9]:
features = list(walmart_df.columns.values)
del features[0]

In [10]:
features

['Weekday',
 '1-HR PHOTO',
 'ACCESSORIES',
 'AUTOMOTIVE',
 'BAKERY',
 'BATH AND SHOWER',
 'BEAUTY',
 'BEDDING',
 'BOOKS AND MAGAZINES',
 'BOYS WEAR',
 'BRAS & SHAPEWEAR',
 'CAMERAS AND SUPPLIES',
 'CANDY, TOBACCO, COOKIES',
 'CELEBRATION',
 'COMM BREAD',
 'CONCEPT STORES',
 'COOK AND DINE',
 'DAIRY',
 'DSD GROCERY',
 'ELECTRONICS',
 'FABRICS AND CRAFTS',
 'FINANCIAL SERVICES',
 'FROZEN FOODS',
 'FURNITURE',
 'GIRLS WEAR, 4-6X  AND 7-14',
 'GROCERY DRY GOODS',
 'HARDWARE',
 'HEALTH AND BEAUTY AIDS',
 'HOME DECOR',
 'HOME MANAGEMENT',
 'HORTICULTURE AND ACCESS',
 'HOUSEHOLD CHEMICALS/SUPP',
 'HOUSEHOLD PAPER GOODS',
 'IMPULSE MERCHANDISE',
 'INFANT APPAREL',
 'INFANT CONSUMABLE HARDLINES',
 'JEWELRY AND SUNGLASSES',
 'LADIES SOCKS',
 'LADIESWEAR',
 'LARGE HOUSEHOLD GOODS',
 'LAWN AND GARDEN',
 'LIQUOR,WINE,BEER',
 'MEAT - FRESH & FROZEN',
 'MEDIA AND GAMING',
 'MENSWEAR',
 'NULL',
 'OFFICE SUPPLIES',
 'OPTICAL - FRAMES',
 'OPTICAL - LENSES',
 'OTHER DEPARTMENTS',
 'PAINT AND ACCESSORIES'

In [11]:
#mytrain, mytest = train_test_split(walmart_df, test_size = .4)

In [12]:
walmart_df_Y=walmart_df['TripType']
walmart_df=walmart_df.drop(['TripType'], axis=1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(walmart_df, walmart_df_Y, random_state=0, stratify =walmart_df_Y)

In [14]:
mytrain_train = X_train
mytest_train = X_test

In [15]:
#mytrain_train = mytrain[features]
#mytest_train = mytest[features]
mytest_train.head()

,Weekday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,...,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS,total_prod,ind_dev,dia_mes,departamentos_visitados,prod_distintos_upc,prod_devueltos
VisitNumber,,,,,,,,,,,,,,,,,,,,,
53982,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,9.0,4,4.0,0.0
103781,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,3.0,1.0,17.0,3,4.0,1.0
125806,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,21.0,2,4.0,0.0
50101,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,14.0,0.0,9.0,3,8.0,0.0
92147,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,15.0,1,3.0,0.0


In [16]:
#dtrain = xgb.DMatrix(np.asarray(mytrain_train), label = np.asarray(mytrain.TripType))
#dtest = xgb.DMatrix(np.asarray(mytest_train), label = np.asarray(mytest.TripType))

In [62]:
#list(mytest_train.columns.values)

In [17]:
dtrain = xgb.DMatrix(np.asarray(mytrain_train), label = np.asarray(y_train))
dtest = xgb.DMatrix(np.asarray(mytest_train), label = np.asarray(y_test))

In [18]:
num_round = 200
param = {'objective': 'multi:softprob', 'num_class':38, 
         'eval_metric': 'mlogloss', "max_delta_step": 5}
watchlist = [(dtrain,'train'), (dtest, 'eval')]

In [19]:
bst = xgb.train(param, dtrain, num_round, watchlist, 
                early_stopping_rounds=3)

[23:34:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=6
[23:34:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=6
[23:34:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=6
[23:34:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=6
[23:34:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 0 pruned nodes, max_depth=6
[23:34:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 0 pruned nodes, max_depth=6
[23:34:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=6
[23:34:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=6
[23:34:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_d

In [20]:
my_test = xgb.DMatrix(np.asarray(mytest_train))
test_predictions = bst.predict(my_test)
#log_loss(mytest.TripType, test_predictions).round(5)

In [21]:
log_loss(y_test, test_predictions).round(5)

0.87565999999999999

# Base test Final


In [71]:
walmart_df_test = feather.read_dataframe('walmart_test.feather')

In [72]:
walmart_df_test.head()

,VisitNumber,Weekday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,...,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS,total_prod,ind_dev,dia_mes,departamentos_visitados,prod_distintos_upc,prod_devueltos
0,1,Friday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,1.0,4,4.0,0.0
1,2,Friday,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,1.0,4,4.0,0.0
2,3,Friday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,1.0,1.0
3,4,Friday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2,1.0,0.0
4,6,Friday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,1.0,1.0


In [73]:
walmart_df_test.shape

(95674, 76)

In [74]:
walmart_df_test['Weekday'] = walmart_df_test['Weekday'].map({"Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4, "Friday": 5, 
                                           "Saturday": 6, "Sunday": 7})

In [75]:
walmart_df_test.head()

,VisitNumber,Weekday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,...,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS,total_prod,ind_dev,dia_mes,departamentos_visitados,prod_distintos_upc,prod_devueltos
0,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,1.0,4,4.0,0.0
1,2,5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,1.0,4,4.0,0.0
2,3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,1.0,1.0
3,4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2,1.0,0.0
4,6,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,1.0,1.0


In [76]:
walmart_df_test = walmart_df_test.set_index('VisitNumber')

In [77]:
walmart_df_test.head()

,Weekday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,...,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS,total_prod,ind_dev,dia_mes,departamentos_visitados,prod_distintos_upc,prod_devueltos
VisitNumber,,,,,,,,,,,,,,,,,,,,,
1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,1.0,4,4.0,0.0
2,5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,1.0,4,4.0,0.0
3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,1.0,1.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2,1.0,0.0
6,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,1.0,1.0


In [78]:
walmart_test = xgb.DMatrix(np.asarray(walmart_df_test))
final_predictions = bst.predict(walmart_test)

In [79]:
#walmart_test.get_label

In [80]:
test_predictions = pd.DataFrame(final_predictions)

In [81]:
test_indexes = walmart_df_test.index

In [82]:
test_predictions = test_predictions.rename(columns = {0:"TripType_3", 1: "TripType_4", 2: "TripType_5", 3: "TripType_6", 
                4: "TripType_7", 5: "TripType_8", 6: "TripType_9", 7: "TripType_12", 8: "TripType_14", 
                9: "TripType_15", 10: "TripType_18", 11: "TripType_19", 12: "TripType_20", 13: "TripType_21",
                14: "TripType_22", 15: "TripType_23", 16: "TripType_24", 17: "TripType_25", 18: "TripType_26", 
                19: "TripType_27", 20: "TripType_28", 21: "TripType_29", 22: "TripType_30", 23: "TripType_31", 
                24: "TripType_32", 25: "TripType_33", 26: "TripType_34", 27: "TripType_35", 28: "TripType_36", 
                29: "TripType_37", 30: "TripType_38", 31: "TripType_39", 32: "TripType_40", 33: "TripType_41", 
                34: "TripType_42", 35: "TripType_43", 36: "TripType_44", 37: "TripType_999"})
test_predictions.insert(0, 'VisitNumber', test_indexes)

In [83]:
test_predictions.head()

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,0.000016,0.000006,0.000056,0.000291,0.004933,0.006007,0.014203,0.000023,0.000013,...,0.000179,0.000214,0.380380,0.113438,0.000127,0.007297,0.013090,0.004286,0.000754,0.005640
1,2,0.000162,0.000044,0.000368,0.002591,0.046968,0.030247,0.013874,0.000103,0.000232,...,0.001071,0.000576,0.062491,0.156502,0.000433,0.011898,0.017868,0.030954,0.001018,0.015778
2,3,0.000023,0.000002,0.000014,0.000099,0.001081,0.003285,0.004092,0.000002,0.000009,...,0.000040,0.000018,0.000071,0.000045,0.000010,0.000057,0.000066,0.000005,0.000006,0.990637
3,4,0.000152,0.000009,0.000086,0.000327,0.005447,0.127543,0.714523,0.000012,0.000020,...,0.000240,0.000110,0.000493,0.000244,0.000053,0.000104,0.000521,0.000058,0.000030,0.144540
4,6,0.000023,0.000002,0.000014,0.000099,0.001081,0.003285,0.004092,0.000002,0.000009,...,0.000040,0.000018,0.000071,0.000045,0.000010,0.000057,0.000066,0.000005,0.000006,0.990637


In [84]:
test_predictions.to_csv("xgb.csv", index=False)

In [86]:
import pickle
pickle_out = open("bst_model.pickle","wb")
pickle.dump(bst, pickle_out)
pickle_out.close()

In [87]:
pickle_in = open("bst_model.pickle","rb")
example_dict = pickle.load(pickle_in)

In [88]:
final_predictions_aux = example_dict.predict(walmart_test)